In [ ]:
# Data to Acquire:
# 1. overview of electric cars currently on the market from ev-database.org
# 2. Average car prices for consumers since 1960 (real and nominal)
# 3. US median wage since 1960 (real and nominal)
# 4. EV battery prices since 2013, can also be component prices (Li, Cobalt, Nickel, Aluminum)
# 5. EVs produced yearly since 2013
# 6. EVs sold yearly 2013

In [3]:
# Step 1: imports
from requests import get
from bs4 import BeautifulSoup
import os
import requests
import json
from typing import Dict, List, Optional, Union, cast
import time
import pandas as pd
from datetime import date

In [4]:
### Always Check for the Hidden API When Web Scraping - Youtube Video by John Watson Rooney

### Using REI's site as an example, under section camp electronics

# 1. navigate to site, inspect page elements
# 2. in the inspect page elemnets box, click the network tab in the toolbar at top, and check the box for 'Fetch/XHR' to retrieve the full web request
# 3. navigate to the bottom of the target page, click on 'more' or 'page 2', which will generate more search results
# 4. select the request that contains all of the web page info (page size/items). This result is the body of the scrpaing file we will build.
# 5. Copy the appropriate result with a right click "copy cURL"
# 6. Paste into Insomnia (API tool), and attempt to edit the item counts per page, to reduce aggregate page requests during scraping.
# 7. Once complete with parameter adjustment, click the drop down arrow to the right of the GET request, select 'generate code'
# 8. Select 'Python', copy to clipboard, paste in IDE
# 9. Note: common issues with generated dict:
#     a. quotes: 2x double quotes. fix by replacing the outer double quotes with single quotes
#     b. clean up the code by removing the empty payload varaible, then from the response variable equation

In [13]:
# do the basics, set up a get(url), check response
url = "https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=9"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
response = requests.get(url, headers)
soup = BeautifulSoup(response.text, 'html.parser')
soup.text[:50]
# functions check complete

'\n\n\n\n\n     \nCompare electric vehicles - EV Database'

In [27]:
def scrapeEVs():

    start = time.time()

    results = []

    url = "https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=all"
    
    headers = {"cookie": "PHPSESSID=sr0qffdk7v17jhqrcu7bohj682; locale=si"}

    while True:
        try:
            response = requests.request("GET", url, headers=headers, timeout=10)
            break
        except:
            print('response unsuccessful')
            time.sleep(15)

    time_taken = time.time() - start

    #following line commented out for scrapes only 1 page in length
    #print(f"\rFetching page {page} of {n-1} at {url}, time per query is {time_taken}", end="")

    soup = BeautifulSoup(response.text, 'html.parser')
    
    make = soup.find(class_="title").text
    model = soup.find(class_="title").text
    accel = soup.find(class_="acceleration").text
    speed = soup.find(class_="topspeed").text
    ev_range = soup.find(class_="erange_real").text
    bev = soup.find(class_="battery").text
    charge = soup.find(class_="fastcharge_speed_print").text
    g_price = soup.find(title="Price in Germany before incentives").text
    d_price = soup.find(title="Price in The Netherlands before incentives").text
    uk_price = soup.find(title="Price in the United Kingdom after incentives").text

    evFacts = {'Make': make,
            'Model': model,
            '0_t0_100': accel,
            'Top_Speed': speed,
            'Range': ev_range,
            'Battery_Capacity': bev,
            'Fast_Charge': charge,
            'German_Price': g_price,
            'Dutch_Price': d_price,
            'UK_Price': uk_price}
    
    results.append(evFacts)

    results_df = pd.DataFrame(results)
    #results_df.to_csv('REIscrape.csv')
    return results_df

In [28]:
# functions check
url = "https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=all"
headers = {"cookie": "PHPSESSID=sr0qffdk7v17jhqrcu7bohj682; locale=si"}
soup = BeautifulSoup(response.text, 'html.parser')
soup.find(class_="title").text

'MG MG4 Electric Long Range'

In [29]:
scrapeEVs()

,Make,Model,0_t0_100,Top_Speed,Range,Battery_Capacity,Fast_Charge,German_Price,Dutch_Price,UK_Price
0,MG MG4 Electric Long Range,MG MG4 Electric Long Range,8.0 sec,160 km/h,360 km,61.7,440 km/h,"€35,990","* €33,000","£28,495"
